In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [2]:
import keras
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
#config.log_device_placement = True  # to log device placement (on which device the operation ran)
                                    # (nothing gets printed in Jupyter, only if you run it standalone)
sess = tf.Session(config=config)
set_session(sess) 

/home/vle/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
from __future__ import print_function

from keras.models import Sequential
from keras.layers import Conv1D, Dense, Dropout, Flatten, BatchNormalization
from keras.callbacks import EarlyStopping, CSVLogger

from loader import load_20news
from custom_layer import MonteCarloLRF, SeparableMonteCarloLRF, SeparableMonteCarloMaxPoolingV2, RandomLRF

import numpy as np
import sklearn as sk
import pickle

In [4]:
top_words=10000
sparse=False
remove_short_documents=True
notebook = 'mcNet_top10k_temptative_42'

In [5]:
(input_shape, nb_classes), (X_train, X_test, Y_train, Y_test), graph_data = \
    load_20news(data_home='data', top_words=top_words, sparse=sparse,
                remove_short_documents=remove_short_documents, verbose=False)

In [6]:
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2])
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2])
num_classes = Y_train.shape[1]

In [7]:
%%time
if False:
  #Process next cell only once
  path = os.path.join('probabilities_' + 
                      'top' + str(top_words) +
                      '_sparse' + str(sparse) +
                      '_removeShorts' + str(remove_short_documents) +
                      '_tfidf_bis.pkl')
  if os.path.isfile(path):
    probabilities = pickle.load(open(path, "rb"), encoding='latin1')

CPU times: user 1 µs, sys: 1 µs, total: 2 µs
Wall time: 4.77 µs


In [8]:
%%time
#if not(os.path.isfile(path)):
if True:
  METRIC = 'cosine'#'euclidean'
  distances = sk.metrics.pairwise.pairwise_distances(graph_data, metric=METRIC, n_jobs=-2)

  # enforce exact zero
  for k in range(distances.shape[0]):
    distances[k,k] = 0.

  # max normalize
  #distances /= distances.max()
  distances /= distances.max(axis=1).reshape((distances.shape[0], 1))

  # use tricube kernel (becaause of flatness around 0)
  probabilities = (1. - np.abs(distances) ** 3) ** 3

  # remove auto connections (which are taken anyway in LRF)
  for k in range(probabilities.shape[0]):
    probabilities[k,k] = 0.

  # normalize proba
  probabilities /= np.sum(probabilities, axis=1).reshape((probabilities.shape[0], 1))
  
  # pickled for later use
  #pickle.dump(probabilities, open(path,"wb"))  

CPU times: user 12.2 s, sys: 1.17 s, total: 13.3 s
Wall time: 12.7 s


In [9]:
if False:
  probabilities = np.ones(probabilities.shape)

  # remove auto connections (which are taken anyway in LRF)
  for k in range(probabilities.shape[0]):
    probabilities[k,k] = 0.

  # renormalize proba
  probabilities /= np.sum(probabilities, axis=-1).reshape((probabilities.shape[0], 1))
  assert ((np.sum(probabilities, axis=-1) - 1) < 0.000001).all()

In [10]:
batch_size = 64
model = Sequential()
model.add(RandomLRF(probabilities, LRF_size=2, filters=1, activation='relu',
                       input_shape=(X_train.shape[1], X_train.shape[2])))
#model.add(BatchNormalization(axis=1))
#model.add(Conv1D(1, kernel_size=1, activation='relu',
#                 padding='same',kernel_initializer='he_uniform'))
#model.add(SeparableMonteCarloMaxPoolingV2(LRF_size=4, new_size=2500))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(500, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
random_lrf_1 (RandomLRF)     (None, 10000, 1)          3         
_________________________________________________________________
dropout_1 (Dropout)          (None, 10000, 1)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 10000)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 500)               5000500   
_________________________________________________________________
dropout_2 (Dropout)          (None, 500)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 20)                10020     
Total params: 5,010,523
Trainable params: 5,010,523
Non-trainable params: 0
_________________________________________________________________


In [11]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

early_stopper = EarlyStopping(min_delta=0.001, patience=2)
csv = CSVLogger(notebook + '_log.csv')

history = model.fit(X_train, Y_train,
                    batch_size=batch_size,
                    epochs=20,
                    verbose=1,
                    callbacks=[early_stopper, csv],
                    validation_data=(X_test, Y_test))
score = model.evaluate(X_test, Y_test, verbose=0)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 10168 samples, validate on 7071 samples
Epoch 1/20
10168/10168 [==============================] - 3s 268us/step - loss: 2.7084 - acc: 0.3892 - val_loss: 2.2444 - val_acc: 0.5622
Epoch 2/20
10168/10168 [==============================] - 2s 214us/step - loss: 1.5518 - acc: 0.7192 - val_loss: 1.3659 - val_acc: 0.6916
Epoch 3/20
10168/10168 [==============================] - 2s 214us/step - loss: 0.8527 - acc: 0.8270 - val_loss: 1.1085 - val_acc: 0.7075
Epoch 4/20
10168/10168 [==============================] - 2s 214us/step - loss: 0.5476 - acc: 0.8821 - val_loss: 1.0314 - val_acc: 0.7184
Epoch 5/20
10168/10168 [==============================] - 2s 214us/step - loss: 0.3787 - acc: 0.9187 - val_loss: 1.0140 - val_acc: 0.7180
Epoch 6/20
10168/10168 [==============================] - 2s 214us/step - loss: 0.2658 - acc: 0.9449 - val_loss: 1.0301 - val_acc: 0.7210
Epoch 7/20
10168/10168 [==============================] - 2s 214us/step - loss: 0.1932 - acc: 0.9641 - val_loss: 1.0424 - v

In [12]:
%%bash
tail mcNet_top10k_temptative_42_log.csv

epoch,acc,loss,val_acc,val_loss
0,0.38916207696395444,2.7084440689777223,0.5621552821888881,2.2443757422690918
1,0.7192171518958337,1.5518386728832987,0.6915570641150655,1.3658840517025994
2,0.8270062943033868,0.852667220177189,0.7075378304828664,1.1084849345871186
3,0.8820810385054252,0.547608327621742,0.7184273793359842,1.0313961966727954
4,0.9186664045466671,0.3787094160480634,0.7180031113249631,1.0140072093490764
5,0.9449252557979617,0.2657878479438912,0.7209729881607615,1.03013252308338
6,0.9641030685438311,0.19321497448263536,0.7204072973023815,1.0423866998709306
